Byzantine Generals' Problem
--

Group of generals, each commanding a portion of the Byzantine army, encircle a city. These generals wish to formulate a plan for attacking the city. In its simplest form, the generals must decide only whether to attack or retreat. The important thing is that every general agree on a common decision, for a halfhearted attack by a few generals would become a rout, and would be worse than either a coordinated attack or a coordinated retreat.

Some of the generals might be treacherous who may not only cast a vote for a suboptimal strategy, they may do so selectively. For instance, if nine generals are voting, four of whom support attacking while four others are in favor of retreat, the ninth general may send a vote of retreat to those generals in favor of retreat, and a vote of attack to the rest.
 
And since some of the generals are physically separated and having to send their votes via messengers who may fail to deliver votes or may forge false votes.
 

Byzantine fault tolerance can be achieved if the loyal (non-faulty) generals have a majority agreement on their strategy. There can be a default vote value given to missing messages. For example, missing messages can be given the value Null. Further, if the agreement is that the Null votes are in the majority, a pre-assigned default strategy can be used (e.g., retreat).

## Implementation
### Vote Type
A Vote type of enum is defined which could hold only attack variable or retreat variable

In [1]:
from aenum import Enum


class Vote(Enum):
    attack = 1
    retreat = 2

    def __int__(self):
        return self.value

    def __str__(self):
        return self.name

### General Object

Each general have three main attributes (variables): ID, Weather A general is loyal or treacherous and two dictionaries that described in follow: 

#### Received votes dictionary
A dictionary (map) which maps generals ID to pair of integers that are number of votes received for that general id to attack or retreat consequently (receivedVotes). This dictionary will be initialized with the constructor with the generals votes to its self id and never get updated for general's it's own id, but will be updated upon receiving votes from other generals for others id.


#### Votes dictionary
Also there is another dictionary (votes) that map's generals id to their optimal strategy based on the number of votes received for attack or retreat for that general

In [2]:
import random


class General(object):
    def __init__(self, ID, vote, loyal):
        self.id = ID
        # dictionary: General_ID=>(#attackVotes, #retreatVotes)
        self.receivedVotes = {
            self.id: (1, 0) if vote == Vote['attack'] else (0, 1)
        }
        # dictionary: General_ID=>Vote
        self.votes = {
            self.id:
            Vote['attack'] if vote == Vote['attack'] else Vote['retreat']
        }
        self.loyal = loyal

    # Given a general's id return it's strategy based on the number of
    # attack or retreat votes recevied for that general so far
    def strategy_of_general(self, id):
        return Vote['attack'] if self.receivedVotes[id][
            0] > self.receivedVotes[id][1] else Vote['retreat']

    # Optimal strategy for this general based on the votes recevied so far
    def optimal_strategy(self):
        nAttacks = 0  # Number in favor of attack
        for _, value in self.votes.items():
            nAttacks += 1 if value == Vote['attack'] else 0
        return Vote['attack'] if nAttacks > (
            len(self.votes) / 2) else Vote['retreat']

    # votes should be passed to this funcion and updated self.receivedVotes and self.votes
    def receive_votes(self, votes):
        for key, value in votes.items():
            if self.id == key:  # ignore general's own vote from received dictionary
                continue
            if key not in self.receivedVotes:
                self.receivedVotes[key] = (0, 0)
            if value == Vote['attack']:
                self.receivedVotes[key] = (self.receivedVotes[key][0] + 1,
                                           self.receivedVotes[key][1])
            else:
                self.receivedVotes[key] = (self.receivedVotes[key][0],
                                           self.receivedVotes[key][1] + 1)
            self.votes[key] = self.strategy_of_general(key)

    def send_votes(self, generals, recursion_level=1):
        for g in generals:
            if self.id == g.id:
                continue
            if self.loyal:
                g.receive_votes(self.votes)
                if recursion_level > 0:  # loyal generals will pass the vote to the other generals
                    g.send_votes(generals, recursion_level - 1)
            else:
                # Pass incorrect vote based on suboptimal solution to other generals
                v = Vote['attack'] if self.optimal_strategy(
                ) == Vote['retreat'] else Vote['retreat']
                g.receive_votes({self.id:v})

    # !common_decision: Common decision according to each general optimal strategy
    # @generals: list of General object
    @staticmethod
    def common_decision(generals):
        nAttacks = 0
        for g in generals:
            nAttacks += 1 if g.optimal_strategy() == Vote['attack'] else 0
        return Vote['attack'] if nAttacks > (
            len(generals) / 2) else Vote['retreat']

    # By calling this method generals will exchange the votes randomly
    @staticmethod
    def exchangeVotes(generals, recursion_level, nExchanges):
        for i in range(nExchanges):
            gid = random.randint(0, len(generals) - 1)
            generals[gid].send_votes(generals, recursion_level)

### Simulation

#### Function initialize_generals()
Used to initialize the generals with random vote for attack or retreat. Their loyalty are randomize with ratio of 1/5 for traitors so loyal generals would have the majority.

#### Other implementation details
Recursion level for message exchange is set to 3, so the message will be passed up to 3 times among loyal generals.
There is another value called nRun which runs the simulation multiple times to get the exact results.

In [3]:
def initialize_generals(nGenerals=50, traitorRatio=30):
    nLoyals = nGenerals - int(nGenerals * traitorRatio / 100)
    generals = []
    for i in range(0, nGenerals):
        newGeneral = General(
            i,
            Vote['attack'] if random.randint(0, 1) == 0 else Vote['retreat'],
            bool(random.randint(0, nLoyals)))
        generals.append(newGeneral)

    return generals


# @nRuns: number of runs
def Simulate_Byzantine_Generals(nRuns=10,
             nGenerals=5,
             traitorRatio=30,
             recursionLevel=3,
             verbose=False):
    sum_consensus = 0.0
    for r in range(nRuns):
        generals = initialize_generals(5)
        General.exchangeVotes(generals, recursionLevel, len(generals) * 10)

        common_decision = General.common_decision(generals)

        conformDecisions = 0

        if verbose:
            print("Common decision is:", common_decision, "\n" + str("=" * 30))
            print("\nID\tVote\tDecision\n" + str("_" * 30))

        for g in generals:
            conformDecisions += 1 if g.optimal_strategy(
            ) == common_decision else 0
            if verbose:
                print(
                    str(g.id) + ("*" if not g.loyal else "") + "\t" +
                    str(g.votes[g.id]) + "\t" + str(g.optimal_strategy()))
                #print(g.receivedVotes)

        consensus = float(conformDecisions) * 100 / len(generals)
        sum_consensus += consensus

        if verbose:
            print("=" * 30)
            print("*: Treacherous Generals.")
            print(
                str(round(consensus, 2)) +
                "% of general's act was conform with the common decision.")
            print()

    avg_consensuality = round(sum_consensus / nRuns, 2)
    if verbose:
        print("Averge consensuality:", str(avg_consensuality) + "%")
    return avg_consensuality

## Samle Run to test correctness
**This is a sample run for 10 run and with 20 generals and 35% generals are traitors with 3 recursion.**

** _ Network Average consensuality resulted in 94.0% _ ** This could change due to the randomness of simulation.

In [4]:
Simulate_Byzantine_Generals(
    nRuns=10, nGenerals=20, traitorRatio=35, recursionLevel=3, verbose=True)

Common decision is: retreat 

ID	Vote	Decision
______________________________
0	retreat	retreat
1*	attack	retreat
2	attack	retreat
3	retreat	retreat
4	retreat	retreat
*: Treacherous Generals.
100.0% of general's act was conform with the common decision.

Common decision is: attack 

ID	Vote	Decision
______________________________
0*	attack	attack
1*	retreat	retreat
2*	attack	attack
3	attack	attack
4	attack	attack
*: Treacherous Generals.
80.0% of general's act was conform with the common decision.

Common decision is: retreat 

ID	Vote	Decision
______________________________
0	retreat	retreat
1	retreat	retreat
2	retreat	retreat
3	attack	retreat
4*	retreat	retreat
*: Treacherous Generals.
100.0% of general's act was conform with the common decision.

Common decision is: attack 

ID	Vote	Decision
______________________________
0	attack	attack
1*	retreat	attack
2*	attack	attack
3	attack	attack
4	attack	attack
*: Treacherous Generals.
100.0% of general's act was conform with the common dec

96.0